## Imports 

In [1]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas

In [2]:
import pandas as pd
import numpy as np
import os
import random
import gzip
import shutil

Install komenti (note that java is required for komenti to run).

In [4]:
%%bash
if ! test -f "komenti/bin/komenti"; then
  if ! test -f "komenti-0.2.0-SNAPSHOT.zip"; then
    echo 'downloading komenti'
    wget https://github.com/reality/komenti/releases/download/0.2.0-SNAPSHOT-1/komenti-0.2.0-SNAPSHOT.zip
  fi
  echo 'extracting komenti'
  unzip komenti-0.2.0-SNAPSHOT.zip
  mv komenti-0.2.0-SNAPSHOT komenti
fi


 
 
 
 

CalledProcessError: Command 'b'if ! test -f "komenti/bin/komenti"; then\n  if ! test -f "komenti-0.2.0-SNAPSHOT.zip"; then\n    echo \'downloading komenti\'\n    wget https://github.com/reality/komenti/releases/download/0.2.0-SNAPSHOT-1/komenti-0.2.0-SNAPSHOT.zip\n  fi\n  echo \'extracting komenti\'\n  unzip komenti-0.2.0-SNAPSHOT.zip\n  mv komenti-0.2.0-SNAPSHOT komenti\nfi\n'' returned non-zero exit status 4294967295.

Test komenti for errors:

In [3]:
%%bash
komenti/bin/komenti query -q test

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
Exception in thread "main" java.lang.reflect.UndeclaredThrowableException
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at jsr166y.ForkJoinTask.getThrowableException(ForkJoinTask.java:504)
	at jsr166y.ForkJoinTask.reportResult(ForkJoinTask.java:567)
	at jsr166y.ForkJoinTask.join(ForkJoinTask.java:611)
	at jsr166y.ForkJoinPool.invoke(ForkJoinPool.java:1492)
	at groovyx.gpars.extra166y.ParallelArrayWithMapping.apply(Parall

In [4]:
# check current directory for DIAGNOSES_ICD and NOTEEVENTS
os.listdir()

['.ipynb_checkpoints',
 'AnnotationFiles',
 'DIAGNOSES_ICD.csv.gz',
 'Install and Setup 001.ipynb',
 'Install and Setup.ipynb',
 'komenti',
 'komenti-0.2.0-SNAPSHOT',
 'komenti-0.2.0-SNAPSHOT.zip',
 'module 6 gp 01-02-2021-002.ipynb',
 'module 6 gp 01-02-2021-004.ipynb',
 'module 6 gp 01-02-2021-005.ipynb',
 'module 6 gp 29-01-2021-001.ipynb',
 'module 6 gp 29-01-2021-004.ipynb',
 'module_6_gp02-02-2021-001.ipynb',
 'module_6_gp04-02-2021-001.ipynb',
 'module_6_gp04-02-2021-002.ipynb',
 'NOTEEVENTS.csv.gz',
 'README.md',
 'TextFiles',
 'text_175749.txt',
 'text_183350.txt']

In [4]:
with gzip.open('DIAGNOSES_ICD.csv.gz') as g:
    D_df = pd.read_csv(g)

In [5]:
with gzip.open('NOTEEVENTS.csv.gz') as f:
    NE_df = pd.read_csv(f)

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Tidy up data 

In [6]:
# keep only primary diagnoses (SEQ_NUM = 1.0) so that there is only one entry per patient 

D_df_tidy = D_df.loc[(D_df['SEQ_NUM']==1.0),]

## Sample of 1000 admissions 

In [7]:
# set seed 

random.seed(30)

In [8]:
# loop to select 1000 random, unique admissions 

sample_id = []

while len(set(sample_id)) < 1000 :
    
    i = D_df_tidy.sample(n=1, replace = False).index
        
    if i.isin(NE_df['HADM_ID']) :
        
        if not i.isin(sample_id) :
            
            sample_id.append(D_df_tidy.loc[i[0], 'HADM_ID'])


## Extract and save text for sample patients 

Create a directory and write the concatenated text for each patient, in files named by admission ID, into it.

In [9]:
# IF YOU ARE RUNNING THIS CODE MULTIPLE TIMES delete the existing files 
# and repopulate with new sample 

if os.path.exists('TextFiles'):
    shutil.rmtree('TextFiles')

In [10]:
# create directory for storing text files

if not os.path.exists('TextFiles'):
    os.makedirs('TextFiles')

In [11]:
# for each patient in sample, colect text and write into a .txt file inside TextFiles directory 

path = os.getcwd()

for id in sample_id:
    outfile = path + '/TextFiles/' + "text_{}.txt".format(id)
    
    file = open(outfile,"a+")
    
    text = NE_df.loc[NE_df['HADM_ID'] == id,'TEXT'] 
    
    for i in text:
        file.write(i) 
    
    file.close() 

## Apply Komenti annotation to the text

Load in the HPO ontology terms to the working directory:

In [12]:
%%bash

wget https://raw.githubusercontent.com/reality/synonym_expansion_validation/master/hpo/unexpanded_all.txt


--2021-02-08 17:44:26--  https://raw.githubusercontent.com/reality/synonym_expansion_validation/master/hpo/unexpanded_all.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3047531 (2.9M) [text/plain]
Saving to: ‘unexpanded_all.txt’

     0K .......... .......... .......... .......... ..........  1% 8.47M 0s
    50K .......... .......... .......... .......... ..........  3% 8.73M 0s
   100K .......... .......... .......... .......... ..........  5% 15.8M 0s
   150K .......... .......... .......... .......... ..........  6% 11.8M 0s
   200K .......... .......... .......... .......... ..........  8% 10.6M 0s
   250K .......... .......... .......... .......... .......... 10% 16.2M 0s
   300K .......... .......... .......... .......... .......... 11% 11.7M 0s
   350K .......... .......... .......... .

Annotate the text files with komenti, using the HPO terms in the 'unexpanded_all.txt' file downloaded.

In [14]:
%%bash
komenti/bin/komenti annotate -l unexpanded_all.txt -t TextFiles/ --out annotations.txt  --threads 40 --verbose


[annotate, -l, unexpanded_all.txt, -t, TextFiles/, --out, annotations.txt, --threads, 40, --verbose]
Initialising annotators
[tokenize, ssplit, pos, lemma, ner, regexner, entitymentions, depparse]
Annotating 1000 files ...
1/1000
2/1000
3/1000
4/1000
5/1000
6/1000
7/1000
8/1000
9/1000
10/1000
11/1000
12/1000
13/1000
14/1000
15/1000
16/1000
17/1000
18/1000
19/1000
20/1000
21/1000
22/1000
23/1000
24/1000
25/1000
26/1000
27/1000
28/1000
29/1000
30/1000
31/1000
32/1000
33/1000
34/1000
35/1000
36/1000
37/1000
38/1000
39/1000
40/1000
41/1000
42/1000
43/1000
44/1000
45/1000
46/1000
47/1000
48/1000
49/1000
50/1000
51/1000
52/1000
53/1000
54/1000
55/1000
56/1000
57/1000
58/1000
59/1000
60/1000
61/1000
62/1000
63/1000
64/1000
65/1000
66/1000
67/1000
68/1000
69/1000
70/1000
71/1000
72/1000
73/1000
74/1000
75/1000
76/1000
77/1000
78/1000
79/1000
80/1000
81/1000
82/1000
83/1000
84/1000
85/1000
86/1000
87/1000
88/1000
89/1000
90/1000
91/1000
92/1000
93/1000
94/1000
95/1000
96/1000
97/1000
98/1000
99

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
